In [1]:
from code.neuron import *
from code.encodingsource import *
from code.hsgs import *
from code.classical_neuron import *
from code.classical_pso import *
from code.sf import *
simulator = Aer.get_backend('qasm_simulator')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

from code.experiment_functions import *

/opt/conda/lib/python3.7/site-packages/qiskit/aqua/__init__.py:86: DeprecationWarning: The package qiskit.aqua is deprecated. It was moved/refactored to qiskit-terra For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('aqua', 'qiskit-terra')


In [2]:
diabetes = pd.read_csv('data_and_results/diabetes/diabetes.csv')

In [3]:
# normalization 0-1
for i in diabetes.columns:
    diabetes[i] = (diabetes[i] - diabetes[i].min()) / (diabetes[i].max() - diabetes[i].min())

In [4]:
# split in test and train data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(diabetes.drop('Outcome',axis=1), diabetes['Outcome'], test_size=0.2)

### Train and Test MLP for Comparison

In [5]:
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn import metrics

In [6]:
parameter_space = {
    'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}

In [7]:
mlp = MLPClassifier(max_iter=100)
clf = GridSearchCV(mlp, parameter_space, n_jobs=-1, cv=3)
clf.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=MLPClassifier(max_iter=100), n_jobs=-1,
             param_grid={'activation': ['tanh', 'relu'],
                         'alpha': [0.0001, 0.05],
                         'hidden_layer_sizes': [(50, 50, 50), (50, 100, 50),
                                                (100,)],
                         'learning_rate': ['constant', 'adaptive'],
                         'solver': ['sgd', 'adam']})

In [8]:
print('Best parameters found:\n', clf.best_params_)

Best parameters found:
 {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (50, 50, 50), 'learning_rate': 'constant', 'solver': 'adam'}


In [9]:
predicted = clf.predict(X_test)

In [10]:
print(metrics.precision_score(y_test, predicted))
print(metrics.accuracy_score(y_test, predicted))
print(metrics.recall_score(y_test, predicted))
print(metrics.f1_score(y_test, predicted))

0.7755102040816326
0.8051948051948052
0.6666666666666666
0.7169811320754716


In [18]:
np.min(clf.cv_results_['mean_test_score'])

0.656352622349753

### Train and Test Classical Neuron for Comparison

In [9]:
from sklearn.linear_model import Perceptron

In [7]:
parameter_space = {
    'penalty': ['l2', 'l1', 'elasticnet']
}

In [10]:
percep = Perceptron()
clf = GridSearchCV(percep, parameter_space, n_jobs=-1, cv=3)
clf.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=Perceptron(), n_jobs=-1,
             param_grid={'penalty': ['l2', 'l1', 'elasticnet']})

In [11]:
print('Best parameters found:\n', clf.best_params_)

Best parameters found:
 {'penalty': 'l2'}


In [12]:
predicted = clf.predict(X_test)

In [13]:
print(metrics.precision_score(y_test, predicted))
print(metrics.accuracy_score(y_test, predicted))
print(metrics.recall_score(y_test, predicted))
print(metrics.f1_score(y_test, predicted))

0.6666666666666666
0.6363636363636364
0.03508771929824561
0.06666666666666667


In [18]:
p_precision_score=[]
p_accuracy_score=[]
p_recall_score=[]
p_f1_score=[]

In [19]:
for pen in ['l2', 'l1', 'elasticnet']:
    for i in range(10):
        percep = Perceptron(penalty=pen)
        percep.fit(X_train, y_train)
        predicted = clf.predict(X_test)
        p_precision_score.append(metrics.precision_score(y_test, predicted))
        p_accuracy_score.append(metrics.accuracy_score(y_test, predicted))
        p_recall_score.append(metrics.recall_score(y_test, predicted))
        p_f1_score.append(metrics.f1_score(y_test, predicted))

In [22]:
print(np.max(p_precision_score))
print(np.max(p_accuracy_score))
print(np.mean(p_accuracy_score))
print(np.max(p_recall_score))
print(np.max(p_f1_score))

0.6666666666666666
0.6363636363636364
0.6363636363636365
0.03508771929824561
0.06666666666666667
